In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
import re

In [2]:
from plasmistry.reactions import (Reactions,
                                  CoefReactions, 
                                  CrosReactions)

In [3]:
plt.rcParams["figure.figsize"] = (12,9)
plt.rcParams["font.size"] = 14

In [4]:
np.set_printoptions(precision=1, linewidth=130, edgeitems=5)
pd.options.display.float_format = '{:,.1e}'.format

# Read reactions

In [5]:
reactions = ['2A + 3B =>C+D',
             '2B =>3C+A',
             '=>2D+3A',
             'A =>2C+D',
             ' C =>']
reactant = [re.split(r"=>", _)[0] for _ in reactions]
product = [re.split(r"=>", _)[1] for _ in reactions]

In [6]:
coef = CoefReactions(reactant=pd.Series(reactant),
                 product=pd.Series(product),           
                 k_str=pd.Series(['Tgas*2',
                 'Te',
                 'EN',
                 '2.0d2',
                 '2.0']))

In [8]:
coef.species

0    A
1    B
2    C
3    D
dtype: object

In [9]:
coef.reactant

0    2A + 3B
1         2B
2           
3          A
4          C
dtype: object

In [10]:
coef._Reactions__prdtsij.toarray()

array([[0, 1, 3, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 3, 0, 2, 0],
       [1, 0, 2, 1, 0]], dtype=int64)

In [11]:
coef._Reactions__rcntsij.toarray()

array([[2, 0, 0, 1, 0],
       [3, 2, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]], dtype=int64)

In [12]:
coef._Reactions__sij.toarray()

array([[-2,  1,  3, -1,  0],
       [-3, -2,  0,  0,  0],
       [ 1,  3,  0,  2, -1],
       [ 1,  0,  2,  1,  0]], dtype=int64)

In [13]:
coef._Reactions__rcnt_index

array([[ 0,  1],
       [ 1, -1],
       [-1, -1],
       [ 0, -1],
       [ 2, -1]], dtype=int64)

In [14]:
coef._Reactions__rcnt_expnt

array([[2, 3],
       [2, 0],
       [0, 0],
       [1, 0],
       [1, 0]], dtype=int64)

In [15]:
coef.k_str

0    Tgas*2
1        Te
2        EN
3     2.0d2
4       2.0
dtype: object

In [16]:
coef.compile_k_str()

In [17]:
coef.set_pre_exec_list(['factor = 1',
                       'density = 3'])

In [18]:
coef.set_rate_const(Tgas_K=1000,EN_Td=1,Te_eV=0)

In [19]:
print(coef)


        
====SPECIES====
        
0    A
1    B
2    C
3    D
dtype: object
        
====REACTIONS====
        
 columns: reactions dH_g[eV] dH_e[eV] k_str rate_const rate
        
          reactions  dH_g[eV]  dH_e[eV] rate_const_str  rate_const rate
0  2A + 3B => C + D   0.0e+00   0.0e+00         Tgas*2     2.0e+03  NaN
1      2B => 3C + A   0.0e+00   0.0e+00             Te     0.0e+00  NaN
2        => 2D + 3A   0.0e+00   0.0e+00             EN     1.0e+00  NaN
3       A => 2C + D   0.0e+00   0.0e+00          2.0d2     2.0e+02  NaN
4             C =>    0.0e+00   0.0e+00            2.0     2.0e+00  NaN
        
====PROPERTIES====
        
CLASS : <class 'plasmistry.reactions.reaction_class.CoefReactions'>.
        
Type : k_coefficients related.
        
__4__ species. 
        
__5__ reactions.
        
        
====PRE_EXEC_LIST====
        
factor = 1
density = 3
        
====K_STR_COMPILED====
        
<code object <module> at 0x00000222F4AC81E0, file "<string>", line 1>
      

In [26]:
%timeit coef.set_rate(density=np.arange(5)+1)

9.37 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
print(coef)


        
====SPECIES====
        
0    A
1    B
2    C
3    D
dtype: object
        
====REACTIONS====
        
 columns: reactions dH_g[eV] dH_e[eV] k_str rate_const rate
        
          reactions  dH_g[eV]  dH_e[eV] rate_const_str  rate_const    rate
0  2A + 3B => C + D   0.0e+00   0.0e+00         Tgas*2     2.0e+03 1.6e+04
1      2B => 3C + A   0.0e+00   0.0e+00             Te     0.0e+00 0.0e+00
2        => 2D + 3A   0.0e+00   0.0e+00             EN     1.0e+00 1.0e+00
3       A => 2C + D   0.0e+00   0.0e+00          2.0d2     2.0e+02 2.0e+02
4             C =>    0.0e+00   0.0e+00            2.0     2.0e+00 6.0e+00
        
====PROPERTIES====
        
CLASS : <class 'plasmistry.reactions.reaction_class.CoefReactions'>.
        
Type : k_coefficients related.
        
__4__ species. 
        
__5__ reactions.
        
        
====PRE_EXEC_LIST====
        
factor = 1
density = 3
        
====K_STR_COMPILED====
        
<code object <module> at 0x00000222F4AC81E0, file "<string

In [28]:
coef.get_dn()

array([-32197., -48000.,  16394.,  16202.])